In [4]:
data_ulasan = [
    {'id': 'R1', 'teks': 'PrODuk iNi SANGAT Bagus! kuALItasnya MANTAP.', 'rating': 5},
    {'id': 'R2', 'teks': 'Jelek sEKali, JANGAN beli. SANGat MengECEWakan.', 'rating': 1},
    {'id': 'R3', 'teks': 'bIASa SAJA. Tidak aDa yaNG SpesIAL.', 'rating': 3},
    {'id': 'R4', 'teks': 'Wow LUAR biasa! AMAZING! 10/10', 'rating': 5},
    {'id': 'R5', 'teks': 'Bagus siH, TApi PACKINGNya jelek.', 'rating': 4},
    {'id': 'R6', 'teks': 'BaRAng sesUAI DESKriPSi dan CEPAT sAMpai.', 'rating': 5},
    {'id': 'R7', 'teks': 'Murah tApi MURAHAN, Tidak ReKOMenDASi.', 'rating': 2},
]


In [ ]:
def buat_pencari_keyword(keyword):
    # Mengembalikan closure masseh
    keyword_norm = str(keyword).lower().strip()
    if not keyword_norm:
        raise ValueError("keyword tidak boleh kosong")

    def pencari(teks):
        if teks is None:
            return False
        return keyword_norm in str(teks).lower()

    return pencari

cari_bagus = buat_pencari_keyword("bagus")
cari_jelek = buat_pencari_keyword("jelek")

sentimen_positif = [item["teks"] for item in data_ulasan if cari_bagus(item["teks"])]
sentimen_negatif = [item["teks"] for item in data_ulasan if cari_jelek(item["teks"])]

print(f"sentimen positif = {sentimen_positif}")
print(f"sentimen negatif = {sentimen_negatif}")

sentimen positif = ['Wow LUAR biasa! AMAZING! 10/10']
sentimen negatif = ['Jelek sEKali, JANGAN beli. SANGat MengECEWakan.', 'Bagus siH, TApi PACKINGNya jelek.']


In [ ]:
from functools import wraps


def sanitasi_teks(func):
    @wraps(func)
    def wrapper(teks, *args, **kwargs):
        teks_bersih = "" if teks is None else str(teks).lower().strip()
        return func(teks_bersih, *args, **kwargs)

    return wrapper


@sanitasi_teks
def hitung_jumlah_kata(teks):
    print(f"DEBUG: Menghitung kata dari: '{teks}'")
    return len(teks.split())


for ulasan in data_ulasan:
    jumlah = hitung_jumlah_kata(ulasan["teks"])
    print(f"jumlah kata = {jumlah}")

DEBUG: Menghitung kata dari: 'produk ini sangat bagus! kualitasnya mantap.'
jumlah kata = 6
DEBUG: Menghitung kata dari: 'jelek sekali, jangan beli. sangat mengecewakan.'
jumlah kata = 6
DEBUG: Menghitung kata dari: 'biasa saja. tidak ada yang spesial.'
jumlah kata = 6
DEBUG: Menghitung kata dari: 'wow luar biasa! amazing! 10/10'
jumlah kata = 5
DEBUG: Menghitung kata dari: 'bagus sih, tapi packingnya jelek.'
jumlah kata = 5
DEBUG: Menghitung kata dari: 'barang sesuai deskripsi dan cepat sampai.'
jumlah kata = 6
DEBUG: Menghitung kata dari: 'murah tapi murahan, tidak rekomendasi.'
jumlah kata = 5
